In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

In [50]:
# Читаем
# Там разделитель \t и нет имен столбцов
data_train = pd.read_csv('train.txt', delimiter='\t', header=0, names=['name', 'group'])
data_validate = pd.read_csv('test.txt', delimiter='\t', header=0, names=['name'])

# В нижний регистр
data_train['name'] = data_train['name'].str.lower()
#data_train['group'] = data_train['group'].str.lower()

data_validate['name'] = data_validate['name'].str.lower()

data_train

,name,group
0,державка 30531402 mapal,Резцы
1,"кабель кпсввнг-ls 1х2х0,75",Кабельная продукция
2,"трубка электроизоляционная ткр ф16,0мм",Изделия электроустан
3,"лента конвейер 2,1-1000-тк-200-2-5/2",ИзделияРезино-технич
4,соединение быстроразъемное hs-ss-e-0100,Запчасти
...,...,...
23967,"фреза шпоночная ц/х 8,0",Фрезы
23968,кирпич керам полнотел одинарный м200,МатерСтроительные
23969,"клеймо тв спл 122""ф"" вк15",Инструменты
23970,элемент питания saft ls 14250/std 1/2aa,Запчасти


In [51]:
# Удаляем всё, что не буквы и не цифры
#data_train['name'] = data_train['name'].str.replace('[^a-zA-Zа-яА-Я1-9 ]', ' ')
#data_validate['name'] = data_validate['name'].str.replace('[^a-zA-Zа-яА-Я1-9 ]', ' ')
data_train['name'] = data_train['name'].str.replace('[^a-zA-Zа-яА-Я]', ' ')
data_validate['name'] = data_validate['name'].str.replace('[^a-zA-Zа-яА-Я]', ' ')

data_train

,name,group
0,державка mapal,Резцы
1,кабель кпсввнг ls х х,Кабельная продукция
2,трубка электроизоляционная ткр ф мм,Изделия электроустан
3,лента конвейер тк,ИзделияРезино-технич
4,соединение быстроразъемное hs ss e,Запчасти
...,...,...
23967,фреза шпоночная ц х,Фрезы
23968,кирпич керам полнотел одинарный м,МатерСтроительные
23969,клеймо тв спл ф вк,Инструменты
23970,элемент питания saft ls std aa,Запчасти


In [52]:
# Удаляем слова короче определенного количества символов

min_len = 2
data_train['name'] = data_train['name'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) >= min_len))
data_validate['name'] = data_validate['name'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) >= min_len))

data_train

,name,group
0,державка mapal,Резцы
1,кабель кпсввнг ls,Кабельная продукция
2,трубка электроизоляционная ткр мм,Изделия электроустан
3,лента конвейер тк,ИзделияРезино-технич
4,соединение быстроразъемное hs ss,Запчасти
...,...,...
23967,фреза шпоночная,Фрезы
23968,кирпич керам полнотел одинарный,МатерСтроительные
23969,клеймо тв спл вк,Инструменты
23970,элемент питания saft ls std aa,Запчасти


In [49]:
# Попробуем кириллицу оттранслитить

dictionary = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
      'ж':'zh','з':'z','и':'i','й':'i','к':'k','л':'l','м':'m','н':'n',
      'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'h',
      'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
      'ю':'u','я':'ya'}

for cyr, lat in dictionary.items():
    data_train['name'] = data_train['name'].str.replace(cyr, lat)
    data_validate['name'] = data_validate['name'].str.replace(cyr, lat)
    
data_train['name']

0                            derzhavka mapal
1                           kabel kpsvvng ls
2        trubka elektroizolyacionnaya tkr mm
3                          lenta konveier tk
4            soedinenie bystrorazemnoe hs ss
                        ...                 
23967                     freza shponochnaya
23968       kirpich keram polnotel odinarnyi
23969                       kleimo tv spl vk
23970        element pitaniya saft ls std aa
23971              truba bshht precizionnaya
Name: name, Length: 23972, dtype: object

In [53]:
# Проверяем, не появилось ли пустых строк. Спойлер - появились
print(data_train[data_train['name'] == ''])
print('---------------------------------------------')
print(data_validate[data_validate['name'] == ''])

Empty DataFrame
Columns: [name, group]
Index: []
---------------------------------------------
Empty DataFrame
Columns: [name]
Index: []


In [54]:
# Длина самой длинной строки
# Столько символов должна будет принимать нейронка, строки короче дополним пробелами
max_str_len = data_train['name'].str.len().max()
#print(max_str_len)

def str_extension(strings, new_len):    
    for i, str_i in enumerate(strings):
        if len(str_i) < new_len:
            strings[i] = str_i + ' '*(new_len - len(str_i))
    return strings

#text_train = str_extension(data_train['name'].tolist(), max_str_len)
#text_validate = str_extension(data_validate['name'].tolist(), max_str_len)

# Не будем удлинять, юзнем потом padsequence
text_train = data_train['name'].tolist()
text_validate = data_validate['name'].tolist()

max_str_len

40

In [55]:
# Превращаем тексты в последовательности чисел, соответствующих символам
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#tokenizer = Tokenizer(num_words=num_characters, char_level=True)
tokenizer = Tokenizer(char_level=True, filters='')  # токенизируем на уровне символов
tokenizer.fit_on_texts(data_train['name'].tolist())  # формируем токены на основе частотности в нашем тексте

# Здесь каждую строку в лист чисел
X_train = tokenizer.texts_to_sequences(text_train)
X_validate = tokenizer.texts_to_sequences(text_validate)

# Выравниваем длину, добавляя нули первыми
X_train = pad_sequences(X_train, maxlen=max_str_len)
X_validate = pad_sequences(X_validate,  maxlen=max_str_len)

# Количество уникальных символов, которые мы тут имеем
num_characters = len(tokenizer.word_index)
print('Количество уникальных символов:', num_characters)
X_train.shape

Количество уникальных символов: 59


(23972, 40)

In [56]:
### !!! Это нужно, если на входе нет embedded-слоя. Он кушает прямо последовательности чисел

# Теперь надо каждое число превратить в OHE-массив с длиной num_characters

def sequences_tp_ohe_array(sequences, num_words):
    ohe_arr = np.ndarray((sequences.shape[0], sequences.shape[1], num_words))
    for i, sequence in enumerate(sequences):
        for j, val in enumerate(sequence):
            temp = np.zeros(num_words)
            if val:
                temp[val-1] = 1
            ohe_arr[i, j] = temp
    return ohe_arr

#X_train = sequences_tp_ohe_array(X_train, num_characters)
#X_validate = sequences_tp_ohe_array(X_validate, num_characters)

# Такая функция уже есть, как не трудно было догадаться
from tensorflow.keras.utils import to_categorical

#X_train = to_categorical(X_train, num_classes=num_characters+1)
#X_validate = to_categorical(X_validate, num_classes=num_characters+1)

X_train.shape

(23972, 40)

In [57]:
# Заменим категории на другой OHE
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_train = enc.fit_transform(data_train['group'].to_numpy().reshape(-1, 1)).toarray()
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
# Количество уникальных классов
n_categories = enc.categories_[0].shape[0]
print(n_categories)

96


In [59]:
# Попробуем простенькую модельку
import keras as k

model = k.models.Sequential()
#model.add(k.layers.Input((max_str_len, num_characters)))
model.add(k.layers.Embedding(num_characters+1, 128, input_length = max_str_len))
model.add(k.layers.GRU(128, return_sequences=True))
model.add(k.layers.GRU(128))
model.add(k.layers.Dense(n_categories, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 40, 128)           7680      
_________________________________________________________________
gru (GRU)                    (None, 40, 128)           99072     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense_4 (Dense)              (None, 96)                12384     
Total params: 218,208
Trainable params: 218,208
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Учимся
history = model.fit(X_train, y_train, batch_size=200, epochs=20, validation_split = 0.1)

Epoch 1/20
108/108 [==============================] - 32s 258ms/step - loss: 3.8983 - accuracy: 0.2312 - val_loss: 3.1523 - val_accuracy: 0.3399
Epoch 2/20
108/108 [==============================] - 29s 272ms/step - loss: 3.0531 - accuracy: 0.3466 - val_loss: 2.7372 - val_accuracy: 0.3891
Epoch 3/20
108/108 [==============================] - 28s 261ms/step - loss: 2.6391 - accuracy: 0.4102 - val_loss: 2.4065 - val_accuracy: 0.4529
Epoch 4/20
108/108 [==============================] - 27s 247ms/step - loss: 2.3147 - accuracy: 0.4632 - val_loss: 2.1330 - val_accuracy: 0.5058
Epoch 5/20
108/108 [==============================] - 29s 271ms/step - loss: 2.0395 - accuracy: 0.5144 - val_loss: 1.9038 - val_accuracy: 0.5550
Epoch 6/20
108/108 [==============================] - 29s 270ms/step - loss: 1.8232 - accuracy: 0.5636 - val_loss: 1.7391 - val_accuracy: 0.5784
Epoch 7/20
108/108 [==============================] - 29s 265ms/step - loss: 1.6168 - accuracy: 0.6031 - val_loss: 1.6056 - val_ac

In [61]:
# Записываем веса в файл
model.save('.\\simple_gru_saved')

INFO:tensorflow:Assets written to: .\simple_gru_saved\assets


INFO:tensorflow:Assets written to: .\simple_gru_saved\assets


In [62]:
# Читаем веса из файла
model = k.models.load_model('.\\simple_gru_saved')